# Statement Extractor

This Notebook guides you through the *Statement Extractor* module.

In [1]:
import pandas as pd
import os

from src.utils import pdf_parser
from src import segmenter, detect_claims, highlight_pdf
from config import SAMPLE_PDF_PATH, MODEL_NAME, MODEL_WEIGHTS_PATH, SPACY_DATA_PATH
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
pd.options.mode.chained_assignment = None  # default='warn'

## 0. Load data

In [2]:
text = pdf_parser.pdf_to_text(SAMPLE_PDF_PATH)  # load all text from pdf

In [3]:
parse = pdf_parser.parse_text(text)  # parse pdf with specific document parser

## 1. Create segments

In [4]:
title = parse["title"]
date = parse["date"]
passages = parse["passages"]  # this document has natural passages

In [5]:
doc = []  # create empty doc

In [6]:
passage_id = 0
make_segments = True
for passage in passages:
    if make_segments:
        segments = segmenter.split_segments(passage.get("text"), segment_len=2)
    else:
        sentences = segmenter.split_sentences(passage.get("text"))
        segments = [{"sentence": sentence} for sentence in sentences]
    
    for segment in segments:
        segment["passage_id"] = passage_id
        doc.append(segment)
    
    passage_id += 1

### 1.1 Load to table

In [7]:
doc_table = pd.DataFrame(doc)

In [8]:
doc_table.head()

,segment_id,sentence,passage_id
0,0,Guten Mittag liebe Journalistinnen und Journal...,0
1,0,Herzlich willkommen hier zu unserem virtuellen...,0
2,1,"Mein Name ist Bastian Zimmermann, und ich bin ...",0
3,2,"Ich freue mich, heute auch unsere drei Experte...",0
4,3,"Man hört immer wieder, wie wichtig Quantentech...",0


### 1.2 Add Metadata

In [9]:
doc_table["title"] = title
doc_table["date"] = date

In [10]:
doc_table["speaker"] = None
doc_table["timestamp"] = None

for idx, passage in enumerate(passages):
    speaker = passage.get("speaker")
    timestamp = passage.get("timestamp")
    
    doc_table['speaker'].loc[doc_table['passage_id'] == idx] = speaker
    doc_table['timestamp'].loc[doc_table['passage_id'] == idx] = timestamp


In [11]:
doc_table.head()

,segment_id,sentence,passage_id,title,date,speaker,timestamp
0,0,Guten Mittag liebe Journalistinnen und Journal...,0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00]
1,0,Herzlich willkommen hier zu unserem virtuellen...,0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00]
2,1,"Mein Name ist Bastian Zimmermann, und ich bin ...",0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00]
3,2,"Ich freue mich, heute auch unsere drei Experte...",0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00]
4,3,"Man hört immer wieder, wie wichtig Quantentech...",0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00]


## 2. Detect main concept
Topic Sentence similarity by sentence embedding (dot product of word embeddings)

In [12]:
from src.main_concept import sentence_title_similarity, wikified_article_score, wikify
from config import TAGME_TOKEN, DANDELION_TOKEN

In [13]:
similarities = sentence_title_similarity(text, title, doc_table["sentence"].to_list())

In [14]:
doc_table["title_sentence_similaritie"] = similarities

Score as the sum of probabilitys from all spots in the title/introduction and sentence

In [15]:
introduction = """Quantencomputer sollen eine der wichtigsten Technologien der Zukunft werden. In der Kryptographie könnten sie bald aktuelle Verschlüsselungsmethoden knacken oder neue Methoden möglich machen. Im Bereich der Materialforschung und -entwicklung sollen sie helfen, Strukturen von Chemikalien, Molekülen oder Medikamenten besser zu berechnen und zu simulieren – oder gar ganz neue Wege ermöglichen. Auch im Bereich von Optimierungsprozessen, Risikoanalysen und Detektion von Produktionsfehlern erhoffen sich Forschende Durchbrüche. Deswegen haben nicht nur Wissenschaftlerinnen und Wissenschaftler, sondern auch Unternehmen ein starkes Interesse an der Weiterentwicklung von Quantencomputern. Große Schlagzeilen machten in letzter Zeit oft Firmen, zum Beispiel Google, das 2019 verkündete, erstmals Quantenüberlegenheit erreicht zu haben [I] – also mit einem Quantencomputer eine Aufgabe gelöst zu haben, die ein klassischer Computer nicht in einem realistischen Zeitrahmen hätte lösen können. Damit die akademische Forschung nicht von der Privatwirtschaft abgehängt wird, haben Politiker in Deutschland und Europa in den letzten Jahren viele Förderprojekte initialisiert. So unterstützt die Bundesregierung den Forschungsbereich mit bis zu zwei Milliarden Euro [II], und das 2018 initialisierte EU-Flaggschiff-Projekt zu Quantencomputern stellt führenden europäischen Forschenden aus dem Bereich Quantentechnologien bis zu eine Milliarde Euro in Aussicht [III]. Das langfristige Ziel ist ein europäisches Quanteninternet. Das daraus entwachsene Projekt OpenSuperQ verfolgt das Ziel, den ersten Quantencomputer in Europa zu entwickeln [IV]. Doch kommen diese Förderungen da an, wo sie gebraucht werden? Wie steht Europa im internationalen Vergleich dar? Was passiert momentan in der Forschung zu Quantentechnologien? Welche Hoffnungen sind berechtigt, welche Befürchtungen begründet? Was wird in naher Zukunft passieren, wann gibt es Durchbrüche? Und welche Auswirkungen dieser Technologien sind zu erwarten? Diese und vor allem Ihre Fragen beantworteten drei Experten in einem 50-minütigen virtuellen Press Briefing."""

In [16]:
title_wikifyed = wikify(text=title, service="tagme", token=TAGME_TOKEN)
intor_wikifyed = wikify(text=introduction, service="tagme", token=TAGME_TOKEN)


In [17]:
title_article_ids = {article["id"]for article in title_wikifyed}  # get all article ids from title
intro_article_ids = {article["id"]for article in intor_wikifyed}  # get all article ids from introduction

In [18]:
doc_table["title_wiki_score"] = doc_table.apply(lambda x: wikified_article_score(x["sentence"], title_article_ids), axis=1)

In [19]:
doc_table["introduction_wiki_score"] = doc_table.apply(lambda x: wikified_article_score(x["sentence"], intro_article_ids), axis=1)

In [20]:
doc_table.sort_values(by="introduction_wiki_score", ascending=False).head()

,segment_id,sentence,passage_id,title,date,speaker,timestamp,title_sentence_similaritie,title_wiki_score,introduction_wiki_score
1,0,Herzlich willkommen hier zu unserem virtuellen...,0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00],0.259937,1.000000,1.464395
152,1,Ist zum Beispiel der Quantencomputer von Googl...,14,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:21:52],0.695814,1.000000,1.223249
347,0,"Ein Aspekt der Tatsache, dass ein Quantencompu...",54,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Frank Wilhelm-Mauch,[00:50:01],0.556523,1.015368,1.144498
53,2,"Herr Wilhelm-Mauch, mit OpenSuperQ sind Sie ja...",4,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:05:28],0.344153,1.043478,1.046679
163,1,Da (sind die Ziele) grob zusammengefasst inner...,16,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:22:58],0.695889,1.000000,1.046156


## 3. Detect claim sentences

In [21]:
detector = detect_claims.claim_detector(MODEL_NAME, MODEL_WEIGHTS_PATH)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.
Some layers from the model checkpoint at deepset/gbert-base were not used when initializing TFDistilBertForSequenceClassification: ['nsp___cls', 'mlm___cls', 'bert']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSe

In [22]:
doc_table["claim"] = doc_table.apply(lambda x: detector.is_claim(x["sentence"]), axis=1)

In [23]:
doc_table.head()

,segment_id,sentence,passage_id,title,date,speaker,timestamp,title_sentence_similaritie,title_wiki_score,introduction_wiki_score,claim
0,0,Guten Mittag liebe Journalistinnen und Journal...,0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00],0.195183,0.0,0.000000,False
1,0,Herzlich willkommen hier zu unserem virtuellen...,0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00],0.259937,1.0,1.464395,False
2,1,"Mein Name ist Bastian Zimmermann, und ich bin ...",0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00],0.004642,0.0,0.000000,False
3,2,"Ich freue mich, heute auch unsere drei Experte...",0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00],0.254553,0.0,0.004200,False
4,3,"Man hört immer wieder, wie wichtig Quantentech...",0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00],0.499324,1.0,1.008076,True


## 4. Return statements

### 4.1 Highlight claim sentences

In [24]:
relevant_claims = doc_table[(doc_table["claim"]==True)&(doc_table["speaker"]!="Moderator")]

In [25]:
claim_sentences = relevant_claims["sentence"].to_list()

In [26]:
highlight_pdf.highlight_text(claim_sentences, SAMPLE_PDF_PATH, "claims.pdf", color=["green", "yellow"])

Warning, sentence not found: "3 Was man auch schon sieht beim Übergang von einem Experiment zu einem richtigen Computer und zu einem Großgerät ist diese ganze Pluralität von Plattformen."
Warning, sentence not found: "Das heißt also, Quantenmaschinen bauen, die es in der Größenordnung von vielleicht einigen zehn, mehreren hundert Quantenbits (geben) wird, die noch nicht fehlerkorrigiert sind, aber auf denen man doch schon 4 Quantenrechnungen machen kann."
Warning, sentence not found: "Allein in Deutschland gibt es zum Beispiel zwei Exzellenzcluster, einen im Rheinland und einen in 8 Würzburg, die sich damit beschäftigen."
Warning, sentence not found: "Wenn man die Ionen und die Supraleiter vergleicht: Die Ionen haben eine unglaublich geringe Fehlerrate und die Supraleiter haben eine etwas höhere Taktfrequenz, die langsamste Operation ist in Nanosekunden gemessen schneller und es kommt dann wirklich auf die Anwendung an, was wie 10 wichtig ist."
Warning, sentence not found: "Ein Aspekt 

### 4.2 Highlight statements

In [27]:
c = "yellow"
highlight_pdf.highlight_text([], SAMPLE_PDF_PATH, "statements.pdf")

passage_id_old = None
segment_id_old = None

for row in relevant_claims.iterrows():
    passage_id = row[1]["passage_id"]
    segment_id = row[1]["segment_id"]

    if passage_id_old==passage_id and segment_id_old == segment_id:
        continue
    else:
        sentences = doc_table[(doc_table["passage_id"]==passage_id)&(doc_table["segment_id"]==segment_id)]["sentence"].to_list()
        highlight_pdf.highlight_text(sentences, "statements.pdf", "statements.pdf", color=c)
        passage_id_old = passage_id
        segment_id_old = segment_id

        if c == "yellow":
            c = "red"
        else:
            c = "yellow"
        

Warning, sentence not found: "3 Was man auch schon sieht beim Übergang von einem Experiment zu einem richtigen Computer und zu einem Großgerät ist diese ganze Pluralität von Plattformen."
Warning, sentence not found: "Das heißt also, Quantenmaschinen bauen, die es in der Größenordnung von vielleicht einigen zehn, mehreren hundert Quantenbits (geben) wird, die noch nicht fehlerkorrigiert sind, aber auf denen man doch schon 4 Quantenrechnungen machen kann."
Warning, sentence not found: "Allein in Deutschland gibt es zum Beispiel zwei Exzellenzcluster, einen im Rheinland und einen in 8 Würzburg, die sich damit beschäftigen."
Warning, sentence not found: "Wenn man die Ionen und die Supraleiter vergleicht: Die Ionen haben eine unglaublich geringe Fehlerrate und die Supraleiter haben eine etwas höhere Taktfrequenz, die langsamste Operation ist in Nanosekunden gemessen schneller und es kommt dann wirklich auf die Anwendung an, was wie 10 wichtig ist."
Warning, sentence not found: "Ein Aspekt 

### 4.3 Highlight statements with claims

In [29]:
c = "yellow"
highlight_pdf.highlight_text([], SAMPLE_PDF_PATH, "statements_claims.pdf")

passage_id_old = None
segment_id_old = None

for row in relevant_claims.iterrows():
    passage_id = row[1]["passage_id"]
    segment_id = row[1]["segment_id"]

    if passage_id_old==passage_id and segment_id_old == segment_id:
        continue
    else:
        sentences = doc_table[(doc_table["passage_id"]==passage_id)&(doc_table["segment_id"]==segment_id)]["sentence"].to_list()
        highlight_pdf.highlight_text(sentences, "statements_claims.pdf", "statements_claims.pdf", color=c)
        passage_id_old = passage_id
        segment_id_old = segment_id

        if c == "yellow":
            c = "red"
        else:
            c = "yellow"
        

Warning, sentence not found: "3 Was man auch schon sieht beim Übergang von einem Experiment zu einem richtigen Computer und zu einem Großgerät ist diese ganze Pluralität von Plattformen."
Warning, sentence not found: "Das heißt also, Quantenmaschinen bauen, die es in der Größenordnung von vielleicht einigen zehn, mehreren hundert Quantenbits (geben) wird, die noch nicht fehlerkorrigiert sind, aber auf denen man doch schon 4 Quantenrechnungen machen kann."
Warning, sentence not found: "Allein in Deutschland gibt es zum Beispiel zwei Exzellenzcluster, einen im Rheinland und einen in 8 Würzburg, die sich damit beschäftigen."
Warning, sentence not found: "Wenn man die Ionen und die Supraleiter vergleicht: Die Ionen haben eine unglaublich geringe Fehlerrate und die Supraleiter haben eine etwas höhere Taktfrequenz, die langsamste Operation ist in Nanosekunden gemessen schneller und es kommt dann wirklich auf die Anwendung an, was wie 10 wichtig ist."
Warning, sentence not found: "Ein Aspekt 

In [30]:
highlight_pdf.highlight_text(claim_sentences, "statements_claims.pdf", "statements_claims.pdf", color="green")

Warning, sentence not found: "3 Was man auch schon sieht beim Übergang von einem Experiment zu einem richtigen Computer und zu einem Großgerät ist diese ganze Pluralität von Plattformen."
Warning, sentence not found: "Das heißt also, Quantenmaschinen bauen, die es in der Größenordnung von vielleicht einigen zehn, mehreren hundert Quantenbits (geben) wird, die noch nicht fehlerkorrigiert sind, aber auf denen man doch schon 4 Quantenrechnungen machen kann."
